In [16]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
from gensim import corpora, models, similarities 
from gensim.corpora.mmcorpus import MmCorpus

time: 470 ms


In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 182 ms


In [19]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 771 ms


In [20]:
#%unload_ext autotime

time: 92.4 ms


[autoreload of autotime failed: Traceback (most recent call last):
  File "/home/jshoun01/.virtualenvs/py27/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named autotime
]


In [21]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 335 ms


In [22]:
ls data

data@
time: 803 ms


In [23]:
data_dir='data'

time: 313 ms


## Normalizer

In [ ]:
ag_preprocesser = helper.Normalizer(stem=False\
                                 ,stopword_removal=True\
                                 ,strip_html_tags=True)

time: 724 ms


## Loading Streaming

In [ ]:
post_file=os.path.join(data_dir,'Posts.xml')
posts_corpus = helper.XmlCorpus(post_file
                                ,clip_docs=100000
                                ,body_field='Body'
                                ,normalizer=ag_preprocesser)

In [ ]:
sample_msg='<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes the version, language, and processor architecture that it was built for.  The WinSxS folder is the only location that the component is found on the system, all other instances of the files that you see on the system are \u201cprojected\u201d by hard linking from the component store.  Let me repeat that last point \u2013 there is only one instance (or full data copy) of each version of each file in the OS, and that instance is located in the WinSxS folder.   So looked at from that perspective, the WinSxS folder is really the entirety of the whole OS, referred to as a "flat" in down-level operating systems.  This also accounts for why you will no longer be prompted for media when running operations such as System File Checker (SFC), or when installing additional features and roles.</p>\n  \n  <p>That explains why the folder starts off big, but not why it gets larger over time \u2013 the answer to that question is servicing.   In previous versions of Windows the atomic unit of servicing was the file, in Windows Vista it\u2019s the component.  When we update a particular binary we release a new version of the whole component, and that new version is stored alongside the original one in the component store.  The higher version of the component is projected onto the system, but the older version in the store isn\u2019t touched.  The reason for that is the third part of why the component store gets so large.</p>\n</blockquote>\n\n<p>Lots of other people <a href="http://blog.tiensivu.com/aaron/archives/1306-Demystifying-the-WinSxS-directory-in-Windows-XP,-Vista-and-Server-20032008.html">complaining about the size of the /winsxs folder, too</a>:</p>\n\n<blockquote>\n  <p>A coworker recently received his every two year work laptop refresh and was in the middle of loading his new laptop with Vista. For some reason, his hard drive C: was partitioned to only 25GB and the rest of the space reserved for a much larger drive D:.</p>\n  \n  <p>Anyway, after loading many Windows Updates, he was left with hardly any drive space left on C:, and for some reason, the WinSxS directory had ballooned to over 8GB.</p>\n  \n  <p>While I knew what the WinSxS (Windows Side-by-Side) directory was for, I wasn\'t quite sure why that directory would explode in size so fast with normal operating system updates.</p>\n</blockquote>\n\n<p>I don\'t really care about having all these duplicate files for safety, but I <em>do</em> care about the VM exploding due to an out of disk space error. So.. <strong>can /winsxs be safely pared down in any way?</strong> <a href="http://blogs.msdn.com/jonwis/archive/2007/01/02/deleting-from-the-winsxs-directory.aspx">It doesn\'t seem that it can be</a>.. </p>\n\n<blockquote>\n  <p><strong>Administrators should not, for any reason, take it upon themselves to clean out the /winsxs directory</strong> - doing so may prevent Windows Update and MSI from functioning properly afterwards.  Preventing accidental deletion from the directory is accomplished by putting a strong security descriptor on the directory that inherits to its children.</p>\n</blockquote>\n\n<p>If this /winsxs folder gets any larger, I\'m going to have to re-image this virtual machine.. which I <em>really</em> don\'t want to do!</p>\n'

ag_preprocesser.process(sample_msg)[0:5]

In [ ]:
"""
num_documents_processed=0
counter=0
for vector in posts_corpus: # load one vector into memory at a time
    print ("Item %s"%(counter))
    print(vector)
    print()
    counter=counter+1
"""

In [ ]:
print ("Number of docs in collection: %s"%(len(posts_corpus)))

In [ ]:
#gensim.models.LdaMulticore
"""
lda = gensim.models.LdaModel(corpus, num_topics=5, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=10000,
                            passes=100)
"""

num_topics = 20

#lda = gensim.models.LdaModel(posts_corpus, num_topics=5, 
#                            id2word=posts_corpus.dictionary
#                                ,passes=2
#                                )

lda = gensim.models.LdaMulticore(posts_corpus, num_topics=num_topics, 
                            id2word=posts_corpus.dictionary
                                 ,passes=3
                                )

In [ ]:
lda.show_topics(num_topics)

In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

In [ ]:
lda.show_topics()

In [ ]:
model_serialized_path='lda_topic_%s.lda'%(num_topics)
lda.save(model_serialized_path) 

In [ ]:
lda = gensim.models.LdaMulticore.load(model_serialized_path)